### Cell annotation using fluoro-forest from our in-house CODEX dataset

In [1]:
# requires cell_annotation environment
import sys
import os
import io
import json
import importlib
import numpy as np
import collections
import scipy
import sklearn
from pySankey.sankey import sankey

import seaborn as sns
import pandas as pd

import matplotlib
import matplotlib.pyplot as plt
from matplotlib.patches import Polygon
from matplotlib.gridspec import GridSpec

from sklearn.preprocessing import StandardScaler

In [2]:
# reading in util functions:
# notebook directory
current_dir = os.getcwd()

# project directory
root_dir = os.path.abspath(os.path.join(current_dir, '..', '..'))
os.chdir(root_dir)

# for importing utils
sys.path.append(os.path.join(root_dir, 'src', 'functions'))

import annotation_utils
import anno_class
import core_class_test
import classifier_class
import plot_utils

In [3]:
markers = pd.read_csv(os.path.join(root_dir, 'example_data', 'anal_cancer', 'raw', 'channelnames.txt'), header = None)[0]
marker_info = {marker:[i, 0, 255] for i, marker in enumerate(markers)}

# Set up as list if wanting to store more than one TMA core at a time
summaries_dir = os.path.join(root_dir, 'example_data', 'anal_cancer', 'summaries')

core_dict = collections.defaultdict()
# set up in case more than one
for core in ['N-12']:
    core_coords = pd.read_csv(os.path.join(summaries_dir, f'{core}_coords.csv')) # coordinate data
    core_coords.set_index('Object.ID', inplace = True)
    
    core_exprs = pd.read_csv(os.path.join(summaries_dir, f'{core}_expression.csv')) # 
    core_exprs.set_index('Object.ID', inplace = True)
    core_exprs.loc[core_coords.index,'X_coord'] = core_coords['Centroid.X.um'] # default names 
    core_exprs.loc[core_coords.index,'Y_coord'] = -1*core_coords['Centroid.Y.um'] # inverted y, for how images are displayed
    core_image = annotation_utils.read_ome_tiff(os.path.join(root_dir, 'example_data', 'anal_cancer', 'processed', f'{core}.ome.tif'))

    # first segmentation is bounding box of core, Qupath specific workflow
    core_segments = annotation_utils.read_geom_json(os.path.join(root_dir, 'example_data', 'anal_cancer','segmentations', f'{core}.geojson'))[1:]
    core_segs_restr = {}
    
    for feature in core_segments:
        core_segs_restr.update(annotation_utils.transform_geometry(feature)) # converts geom_json object into dictionary of coordinates

    core_dict[core] = core_class_test.core_data(expression_data = core_exprs, 
                                           image = core_image, # tif
                                           segments = core_segs_restr, # restructured segmentations
                                           core = core, # name
                                           marker_info = marker_info) # marker channels / slice

In [4]:
# features of interest (all markers) averaged across entire cell segmentation (Cell_Mean) from Qupath workflow
foi = list(core_exprs.columns[~core_exprs.columns.str.contains('DAPI|orig|Cent|coord|Membr|Nucl|Cyto')])

In [5]:
# include states for reproducibility
for key, core in core_dict.items():
    core.select_features(feats = foi)
    core.run_pca()
    core.run_leiden(resolution = 0.5, random_state = 5) 
    core.run_leiden(resolution = 0.8, random_state = 5) 
    core.approximate_bounds()

n12_core = core_dict['N-12']

Running PCA...


In [6]:
plot_utils.cell_plot(core = n12_core, 
                     figsize = (6,4),
                     plot_type = 'PC',
                     col = 'leiden_0.8', 
                     coloring_type = 'categorical', 
                     size = 3,
                     alpha = 1,
                     palette = 'tab20')


In [7]:
plot_utils.expression_heatmap(core = n12_core, 
                              cluster_col = 'leiden_0.8', 
                              cell_mean_substring = 'Cell_Mean', 
                              cmap = 'vlag', 
                              figsize = (4,6))

In [8]:
plot_utils.plot_segmentation(n12_core, invert = True) # visualizes segmentations as polygons

In [9]:
# defining lineage level markers to guide subsclustering steps
marker_dict = {'immune': 'CD45_Cell_Mean', 
               'cd31_stroma':'CD31_Cell_Mean', 
               'sma_stroma':'SMA_Cell_Mean', 
               'epithelia':'Pan.Cytokeratin_Cell_Mean'}

n12_core.lineage_split(marker_dictionary = marker_dict, random_state = 5)

In [10]:
plot_utils.cell_plot(core = n12_core, 
                     figsize = (6,4),
                     plot_type = 'PC',
                     col = 'kmeans_lineage', 
                     coloring_type = 'categorical', 
                     size = 3,
                     alpha = 1,
                     palette = 'tab10') #

In [11]:
resolution_dict = {'0': 0.5, # immune embedded in epithelia?
                   '1': 0.01, # epithelia
                   '2': 0.5, # immune
                   '3': 0.01} # stroma (SMA high)

n12_core.run_stratified_clustering(resolution_dict = resolution_dict, 
                                  cluster_method = 'leiden')

In [12]:
plot_utils.cell_plot(core = n12_core, 
                     figsize = (6,4),
                     plot_type = 'PC',
                     col = 'stratified_cluster', 
                     coloring_type = 'categorical', 
                     size = 3,
                     alpha = 1,
                     palette = 'tab20')

In [13]:
n12_core.cell_sampler(cluster_col = 'stratified_cluster', 
                     max_sample = 100, 
                     keep_oob = False, 
                     tolerance = 1, 
                     use_fps = False, 
                     random_state = 45)

len(n12_core.sampled_cells)

1485

In [14]:
temp_df = n12_core.plot_df.copy()
temp_df.loc[:,'Sampled'] = False
temp_df.loc[n12_core.sampled_cells,'Sampled'] = True

temp_df = temp_df.sort_values('Sampled')

plt.figure(figsize = (4,3))
sns.scatterplot(data = temp_df,  
                x = 'PC1', 
                y = 'PC2', 
                s = 5,
                hue = 'Sampled', 
                palette = ['lightgray', 'red'])

plt.tight_layout()
plt.xticks([])
plt.yticks([])
plt.show()

In [15]:
# this takes a bit of time to manually define thresholds for viewing

n12_core.marker_info['DAPI'] = [0,5,250]
n12_core.marker_info['PanCK'] = [16,0,30]
n12_core.marker_info['CD31'] = [2,0,20]
n12_core.marker_info['SMA'] = [13,0,30]
n12_core.marker_info['CD45'] = [8,5,100]
n12_core.marker_info['CD3e'] = [22,0,20]
n12_core.marker_info['CD4'] = [6,0,20]
n12_core.marker_info['CD8'] = [12,0,30]
n12_core.marker_info['FOXP3'] = [3,5,20]
n12_core.marker_info['CD14'] = [19,0,20]
n12_core.marker_info['CD68'] = [24,3,20]
n12_core.marker_info['CD163'] = [9,0,15]
n12_core.marker_info['HLA-DR'] = [26,0,15]
n12_core.marker_info['CD56'] = [4,2,15]
n12_core.marker_info['CD20'] = [7,2,20]
n12_core.marker_info['CK17'] = [29,0,15]

n12_core.plot_marker(marker = 'DAPI') # adjust manually above and determine appropriate threshold by viewing on whole core

In [16]:
n12_core.annotations = {}

In [17]:
# cell annotations
n12_core.annotate(show_markers = ['DAPI', 'PanCK', 'CD31', 'SMA',
                                 'CD45', 'CD3e', 'CD4', 'CD8', 
                                 'FOXP3', 'CD14', 'CD68', 'CD163',
                                 'HLA-DR', 'CD56', 'CD20', 'CK17'], 
                   cell_types = ['Epi', 'Endothelia', 'Stroma', 
                                 'CD4_T', 'CD8_T','Treg', 'NK', 'Bcell',
                                 'Myeloid','Other_immune', 'Unknown'])

In [17]:
# annotations can be saved part way through after quitting the application and saved out / read back in
annotations_dir = os.path.join(root_dir, 'example_data', 'anal_cancer', 'temp_annotations')

result = pd.read_csv(os.path.join(annotations_dir, 'N12_annotations.csv'))
result.columns = ['Object.ID', 'Annotation']
result.value_counts('Annotation') # many more annotated than actually used in training, classes balanced later on

result = result.set_index('Object.ID')['Annotation'].to_dict()
n12_core.annotations = result 

# pd.DataFrame.from_dict(result, orient = 'index').to_csv(os.path.join(annotations_dir, 'N12_annotations.csv')) # originally saved out after training


In [18]:
np.random.seed(5)
n12_core.annotations = result

# Initialize defaultdict with list as the default factory
new_dict = collections.defaultdict(list)

# Group IDs by cell type
for ids, cell_type in result.items():
    new_dict[cell_type].append(ids)  # Append IDs to the appropriate cell type list

# Downsample 20 IDs per cell type (or keep all if <20)
downsampled_ids = []
for cell_type, ids in new_dict.items():
    sample_size = min(40, len(ids))  # Use 20 or the total available if fewer than 20
    selected_ids = np.random.choice(ids, size = sample_size, replace = False)
    downsampled_ids.extend(selected_ids.tolist())

# Create final downsampled dictionary
downsampled_annotations = {ids: n12_core.annotations[ids] for ids in downsampled_ids}

n12_core.annotations = downsampled_annotations

In [19]:
len(n12_core.annotations.keys()) # used for training / valudation

284

In [20]:
n12_core.plot_df['annotations'] = 'not_annotated' 

# Convert annotations to a list if they aren't
annotations_list = list(n12_core.annotations.values()) #Convert

#Check that keys exist in index before hand!
keys_in_index = [key for key in n12_core.annotations.keys() if key in n12_core.plot_df.index]

# Assign values based on the keys
n12_core.plot_df.loc[keys_in_index, 'annotations'] = annotations_list[:len(keys_in_index)]  # Slicing to avoid length mismatch


In [21]:
# for consistent plotting 
my_pal = sns.color_palette('tab20').as_hex()

cmap = {'not_annotated':'lightgray', 'Epi':my_pal[0], 
        'Myeloid': my_pal[1], 'Endothelia':my_pal[2],
        'Stroma':my_pal[3], 'CD8_T':my_pal[4],
        'CD4_T':my_pal[5], 'Treg':my_pal[6], 
        'Bcell':my_pal[7]}

In [22]:
collections.Counter(n12_core.annotations.values())

Counter({'Epi': 40,
         'Treg': 40,
         'CD8_T': 40,
         'Myeloid': 40,
         'Stroma': 40,
         'Endothelia': 40,
         'CD4_T': 30,
         'Bcell': 14})

In [23]:
plot_utils.cell_plot(core = n12_core, 
                     figsize = (6,4),
                     plot_type = 'cell', 
                     col = 'annotations', 
                     size = 2, 
                     coloring_type = 'categorical', 
                     color_map = cmap)

In [24]:
my_classifier = classifier_class.classify_cells(core_class = n12_core)

rf_feats = n12_core.expression_data.columns[n12_core.expression_data.columns.str.contains('Cell_Mean')]

In [25]:
cv = my_classifier.k_fold_cross_validation(use_params = rf_feats, 
                                           n_splits = 5, 
                                           random_state = 15, 
                                           use_imbalanced_rf = True, 
                                           use_smote = True, 
                                           n_trees = 200)

cv

,fold,class,f1,precision,recall,accuracy
0,1,Bcell,0.857143,0.750000,1.000000,0.859649
1,1,CD4_T,0.833333,0.833333,0.833333,0.859649
2,1,CD8_T,0.875000,0.875000,0.875000,0.859649
3,1,Endothelia,0.823529,0.777778,0.875000,0.859649
4,1,Epi,0.941176,0.888889,1.000000,0.859649
5,1,Myeloid,0.857143,1.000000,0.750000,0.859649
6,1,Stroma,0.750000,0.750000,0.750000,0.859649
7,1,Treg,0.933333,1.000000,0.875000,0.859649
8,2,Bcell,0.800000,1.000000,0.666667,0.929825
9,2,CD4_T,0.909091,1.000000,0.833333,0.929825


In [26]:
print(np.mean(cv.loc[:,'accuracy'])) # 89.1% +/- 3.6%
print(np.std(cv.loc[:,'accuracy']))

0.8909774436090225
0.03551476608304433


In [27]:
plot_utils.plot_metrics(cv, metrics = ['recall', 'precision', 'f1'])

In [28]:
my_classifier.train(split = None, 
                    use_params = rf_feats, 
                    random_state = 19, 
                    use_imbalanced_rf = True, 
                    use_smote = True)
my_classifier.fit()


Class Counts after filtering classes with < 2 samples:
annotations
Epi           40
Stroma        40
Myeloid       40
Treg          40
Endothelia    40
CD8_T         40
CD4_T         30
Bcell         14
Name: count, dtype: int64

Class counts in y_train after train_test_split:
annotations
Epi           40
Stroma        40
Myeloid       40
Treg          40
Endothelia    40
CD8_T         40
CD4_T         30
Bcell         14
Name: count, dtype: int64

Class counts in y_test after train_test_split:
Series([], Name: count, dtype: int64)
min class count 14

applying smote to full *training* set with knn=5

Class counts in y_train after smote:
annotations
Epi           40
CD4_T         40
Stroma        40
Myeloid       40
Bcell         40
Treg          40
CD8_T         40
Endothelia    40
Name: count, dtype: int64


In [29]:
my_classifier.plot_data['prediction'] = ''
n12_core.plot_df.loc[my_classifier.expression_data.index, 'prediction'] = my_classifier.expression_data['predicted_annotation']

In [30]:
plot_utils.cell_plot(core = n12_core, 
                     plot_type = 'cell', 
                     figsize = (6,4),
                     coloring_type = 'categorical', 
                     color_map = cmap,
                     size = 1,
                     col = 'prediction')

In [31]:
plot_utils.contingency_plot(core = n12_core, 
                            column1 = 'leiden_0.8',
                            column2 = 'prediction')

In [32]:
plot_utils.expression_heatmap(core = n12_core, 
                              cluster_col = 'prediction', 
                              cell_mean_substring = 'Cell_Mean', 
                              cmap = 'vlag', 
                              figsize = (4,6))

In [33]:
plot_utils.expression_heatmap(core = n12_core, 
                              cluster_col = 'leiden_0.8', 
                              cell_mean_substring = 'Cell_Mean', 
                              cmap = 'vlag', 
                              figsize = (4,6))

In [34]:
my_classifier.expression_data['annotations'].value_counts()
my_classifier.expression_data['predicted_annotation'].value_counts()

predicted_annotation
Epi           8911
CD8_T         2434
Myeloid       1260
Treg           444
CD4_T          395
Stroma         366
Endothelia     351
Bcell           81
Name: count, dtype: int64

In [35]:
importlib.reload(classifier_class)
my_classifier = classifier_class.classify_cells(core_class = n12_core)
my_classifier.train(split = 0.5, # half train half test 
                    use_params = rf_feats, 
                    random_state = 19, 
                     use_imbalanced_rf = True)
my_classifier.fit()



Class Counts after filtering classes with < 2 samples:
annotations
Epi           40
Stroma        40
Myeloid       40
Treg          40
Endothelia    40
CD8_T         40
CD4_T         30
Bcell         14
Name: count, dtype: int64

Class counts in y_train after train_test_split:
annotations
Endothelia    20
CD8_T         20
Myeloid       20
Epi           20
Stroma        20
Treg          20
CD4_T         15
Bcell          7
Name: count, dtype: int64

Class counts in y_test after train_test_split:
annotations
Endothelia    20
CD8_T         20
Myeloid       20
Epi           20
Stroma        20
Treg          20
CD4_T         15
Bcell          7
Name: count, dtype: int64
model Accuracy: 0.87

Classification Report:
              precision    recall  f1-score   support

       Bcell       1.00      0.86      0.92         7
       CD4_T       0.70      0.93      0.80        15
       CD8_T       0.94      0.80      0.86        20
  Endothelia       0.79      0.95      0.86        20
         

In [36]:
df = my_classifier.expression_data.loc[:,['annotations', 'predicted_annotation']]
df.loc[:,'leiden_0.8'] = my_classifier.plot_data['leiden_0.8']
df = df.loc[df['annotations'] != '', :]
df = df.loc[my_classifier.plot_data['used_in_training'] == False,:]

In [37]:
# my_classifier.plot_data.loc[:,['used_in_training', 'prediction', 'annotations']].to_csv(os.path.join(root_dir, 'n12_results.csv'))
# df.to_csv(os.path.join(root_dir, 'n12_not_trained.csv'))

In [38]:
plot_utils.contingency_plot(core = None, 
                            column1 = df['annotations'], 
                            column2 = df['predicted_annotation'], 
                           figsize = (3.5,3))

In [39]:
plot_utils.contingency_plot(core = None, 
                            column1 = df['annotations'], 
                            column2 = df['leiden_0.8'], 
                           figsize = (3.5,3))

In [40]:
#leiden mapping (0.8)
expression_key = {'0': 'Myeloid', '1':'Epithelia', '2':'CD8_T', 
                  '3':'Epithelia', '4':'Other', '5':'Epithelia', 
                  '6':'Epithelia', '7':'Epithelia', '8':'Epithelia',
                  '9':'CD4_T/Treg', '10':'Stroma', '11':'Bcell'}

In [41]:
converted = df['leiden_0.8'].map(expression_key)
df.loc[:,'leiden_key'] = converted
plot_utils.contingency_plot(core = None, 
                            column1 = df['annotations'], 
                            column2 = df['leiden_key'], 
                           figsize = (3.5,3))

In [42]:
# full model prediction
my_classifier.plot_data['prediction'] = ''
my_classifier.train(split = None, 
                    use_params = rf_feats, 
                    random_state = 19, 
                    use_imbalanced_rf = True, 
                    use_smote = True)

my_classifier.fit()
n12_core.plot_df.loc[my_classifier.expression_data.index, 'prediction'] = my_classifier.expression_data['predicted_annotation']

n12_core.plot_df.loc[:,'prediction'].value_counts()


Class Counts after filtering classes with < 2 samples:
annotations
Epi           40
Stroma        40
Myeloid       40
Treg          40
Endothelia    40
CD8_T         40
CD4_T         30
Bcell         14
Name: count, dtype: int64

Class counts in y_train after train_test_split:
annotations
Epi           40
Stroma        40
Myeloid       40
Treg          40
Endothelia    40
CD8_T         40
CD4_T         30
Bcell         14
Name: count, dtype: int64

Class counts in y_test after train_test_split:
Series([], Name: count, dtype: int64)
min class count 14

applying smote to full *training* set with knn=5

Class counts in y_train after smote:
annotations
Epi           40
CD4_T         40
Stroma        40
Myeloid       40
Bcell         40
Treg          40
CD8_T         40
Endothelia    40
Name: count, dtype: int64


prediction
Epi           8911
CD8_T         2434
Myeloid       1260
Treg           444
CD4_T          395
Stroma         366
Endothelia     351
Bcell           81
Name: count, dtype: int64

In [43]:
plot_utils.cell_plot(core = n12_core, 
                     color_map = cmap,
                     plot_type = 'cell', 
                     figsize = (6,4),
                     coloring_type = 'categorical', 
                     size = 1,
                     col = 'prediction')

In [44]:
for i,clst in enumerate(np.unique(df['leiden_0.8'])):
    cmap[str(clst)] = my_pal[i+7] # 7 is offset for number used in past cmap

In [45]:
sankey(df['annotations'], 
       df['leiden_0.8'], 
       aspect = 5, fontsize = 10,
       colorDict = cmap, 
       rightColor = False)
plt.tight_layout()
plt.show()